In [ ]:
# Version
# Python 3.6.7 :: Anaconda, Inc.

In [45]:
import pandas as pd
import xlrd 
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import time
#nltk.download('wordnet') # 다운 안 받았을 시 다운받고 한 번 다운받은 뒤에는 필요 x

# Data Loading 

In [95]:
data = pd.read_excel('/home/pirl/Downloads/document_clustering/combined recalls.xls')

In [96]:
pd.DataFrame.head(data)

,recall_date,make_match,year,recall_id,country,model_match,make_id,model_id,txt,country_recall_id
0,2007-01-12,VOLVO,2007,1030.0,AUS,C70,27,1175,THE_WRONG_INFORMATION_HAS_BEEN_PRINTED_ON_THE_...,AUS_10300
1,2007-02-12,HONDA,2007,1035.0,AUS,CIVIC,10,497,THERE_IS_INSUFFICIENT_CLEARANCE_BETWEEN_THE_IN...,AUS_10350
2,2007-02-13,MITSUBISHI,2007,1036.0,AUS,TRITON,19,797,AN_INCORRECT_SEATING_CAPACITY_OF_3_INSTEAD_OF_...,AUS_10360
3,2007-02-16,BENTLEY,2007,1039.0,AUS,ARNAGE,2,44,THERE_IS_THE_POTENTIAL_FOR_A_WHEEL_BOLT_TO_LOO...,AUS_10390
4,2007-02-16,BENTLEY,2007,1039.0,AUS,AZURE,2,45,THERE_IS_THE_POTENTIAL_FOR_A_WHEEL_BOLT_TO_LOO...,AUS_10390


# Text Preprocessing + Tokenizing 

In [97]:
def txt_to_words(raw_txt):
    letters_only = re.sub('[^a-zA-Z]', ' ', raw_txt) # 기호 -> 공백으로 처리 
    lower_case = letters_only.lower() # 소문자만 남기기
    step1 = lower_case.split() # 소문자 단위로 나누기 
    step2 = [w for w in step1 if w not in stopwords.words('english')] # 불용어 제거 
    wordnet_lemmatizer = WordNetLemmatizer() # 어근 추출
    txt = [wordnet_lemmatizer.lemmatize(w) for w in step2] # 전처리한 단어 list에 집어넣기 
    
    return txt     


In [98]:
# All data preprocessing ==> ""Time Spend"" 

txt_after_preprocessing = []

for i in range(len(data['txt'])):
    preprocessed_txt = txt_to_words(data['txt'][i])
    txt_after_preprocessing.append(preprocessed_txt)
    

In [99]:
# Check 
len(txt_after_preprocessing)

12432

In [106]:
txt_after_preprocessing[1]

['insufficient',
 'clearance',
 'intelligent',
 'power',
 'unit',
 'cover',
 'dc',
 'dc',
 'converter',
 'cable',
 'terminal',
 'positioned',
 'rear',
 'side',
 'rear',
 'seat',
 'assembly',
 'excessive',
 'weight',
 'placed',
 'upon',
 'rear',
 'set',
 'back',
 'possibility',
 'short',
 'circuit',
 'occurring',
 'short',
 'circuit',
 'due',
 'ipu',
 'cover',
 'interfering',
 'dc',
 'dc',
 'converter',
 'cable',
 'terminal',
 'short',
 'circuit',
 'cause',
 'main',
 'fuse',
 'blow',
 'resulting',
 'engine',
 'stalling',
 'malfunction',
 'honda',
 'australia',
 'contact',
 'owner',
 'affected',
 'vehicle',
 'mail']

# Word2Vec

In [109]:
# embedding model ==> ""Time Spend""

# 전처리된 txt를 100차원의 벡터로 
# 주변 단어(window)는 앞뒤로 4개까지 
# CPU는 쿼드코어를 쓰고 300번 반복 학습
# 분석방법론은 CBOW와 Skip-Gram 중 후자를 선택


start_time=time.time() # 소요시간 보기 위해

embedding_model = Word2Vec(txt_after_preprocessing, size=100, window = 4, min_count=1, workers=4, iter=300, sg=1)

print("Training Runtime: %0.2f Minutes"%((time.time() - start_time)/60)) # 소요시간

Training Runtime: 3.78 Minutes


In [110]:
# Word2vec test 
# 가장 연관성이 높은 10개의 단어 출력해보기 

print(embedding_model.most_similar(positive=['leak'], topn=10))

[('leakage', 0.7429319620132446), ('leaking', 0.6909024715423584), ('pressurized', 0.6025673151016235), ('occasioned', 0.5544111728668213), ('weepage', 0.540611743927002), ('presence', 0.5399015545845032), ('fuel', 0.5365285873413086), ('therefrom', 0.5346571803092957), ('escaping', 0.525587797164917), ('source', 0.5235475301742554)]


/home/pirl/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


# Similarity between Text 

In [111]:
# Example
embedding_model.n_similarity(txt_after_preprocessing[0], txt_after_preprocessing[1])

/home/pirl/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  


0.5067581352028181

In [113]:
# For all text 
# relations[i] : txt[i]와 12342개의 txt간에 word2vec으로 구한 유사도 
start_time=time.time() # 소요시간 보기 위해

relations = []

for i in range(len(after_preprocessing)):
    relation = []
    for d in range(len(after_preprocessing)):
        result = embedding_model.n_similarity(txt_after_preprocessing[i], txt_after_preprocessing[d])
        relation.append(result)
    print('--> get similarity of txt[%s] is done'%i)
    relations.append(relation)

    
print("Runtime: %0.2f Minutes"%((time.time() - start_time)/60)) # 소요시간

/home/pirl/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  # Remove the CWD from sys.path while we load stuff.


--> get similarity of txt[0] is done
--> get similarity of txt[1] is done
--> get similarity of txt[2] is done
--> get similarity of txt[3] is done
--> get similarity of txt[4] is done
--> get similarity of txt[5] is done
--> get similarity of txt[6] is done
--> get similarity of txt[7] is done
--> get similarity of txt[8] is done
--> get similarity of txt[9] is done
--> get similarity of txt[10] is done
--> get similarity of txt[11] is done
--> get similarity of txt[12] is done
--> get similarity of txt[13] is done
--> get similarity of txt[14] is done
--> get similarity of txt[15] is done
--> get similarity of txt[16] is done
--> get similarity of txt[17] is done
--> get similarity of txt[18] is done
--> get similarity of txt[19] is done
--> get similarity of txt[20] is done
--> get similarity of txt[21] is done
--> get similarity of txt[22] is done
--> get similarity of txt[23] is done
--> get similarity of txt[24] is done
--> get similarity of txt[25] is done
--> get similarity of 

--> get similarity of txt[213] is done
--> get similarity of txt[214] is done
--> get similarity of txt[215] is done
--> get similarity of txt[216] is done
--> get similarity of txt[217] is done
--> get similarity of txt[218] is done
--> get similarity of txt[219] is done
--> get similarity of txt[220] is done
--> get similarity of txt[221] is done
--> get similarity of txt[222] is done
--> get similarity of txt[223] is done
--> get similarity of txt[224] is done
--> get similarity of txt[225] is done
--> get similarity of txt[226] is done
--> get similarity of txt[227] is done
--> get similarity of txt[228] is done
--> get similarity of txt[229] is done
--> get similarity of txt[230] is done
--> get similarity of txt[231] is done
--> get similarity of txt[232] is done
--> get similarity of txt[233] is done
--> get similarity of txt[234] is done
--> get similarity of txt[235] is done
--> get similarity of txt[236] is done
--> get similarity of txt[237] is done
--> get similarity of txt

--> get similarity of txt[424] is done
--> get similarity of txt[425] is done
--> get similarity of txt[426] is done
--> get similarity of txt[427] is done
--> get similarity of txt[428] is done
--> get similarity of txt[429] is done
--> get similarity of txt[430] is done
--> get similarity of txt[431] is done
--> get similarity of txt[432] is done
--> get similarity of txt[433] is done
--> get similarity of txt[434] is done
--> get similarity of txt[435] is done
--> get similarity of txt[436] is done
--> get similarity of txt[437] is done
--> get similarity of txt[438] is done
--> get similarity of txt[439] is done
--> get similarity of txt[440] is done
--> get similarity of txt[441] is done
--> get similarity of txt[442] is done
--> get similarity of txt[443] is done
--> get similarity of txt[444] is done
--> get similarity of txt[445] is done
--> get similarity of txt[446] is done
--> get similarity of txt[447] is done
--> get similarity of txt[448] is done
--> get similarity of txt

--> get similarity of txt[635] is done
--> get similarity of txt[636] is done
--> get similarity of txt[637] is done
--> get similarity of txt[638] is done
--> get similarity of txt[639] is done
--> get similarity of txt[640] is done
--> get similarity of txt[641] is done
--> get similarity of txt[642] is done
--> get similarity of txt[643] is done
--> get similarity of txt[644] is done
--> get similarity of txt[645] is done
--> get similarity of txt[646] is done
--> get similarity of txt[647] is done
--> get similarity of txt[648] is done
--> get similarity of txt[649] is done
--> get similarity of txt[650] is done
--> get similarity of txt[651] is done
--> get similarity of txt[652] is done
--> get similarity of txt[653] is done
--> get similarity of txt[654] is done
--> get similarity of txt[655] is done
--> get similarity of txt[656] is done
--> get similarity of txt[657] is done
--> get similarity of txt[658] is done
--> get similarity of txt[659] is done
--> get similarity of txt

...

--> get similarity of txt[11473] is done
--> get similarity of txt[11474] is done
--> get similarity of txt[11475] is done
--> get similarity of txt[11476] is done
--> get similarity of txt[11477] is done
--> get similarity of txt[11478] is done
--> get similarity of txt[11479] is done
--> get similarity of txt[11480] is done
--> get similarity of txt[11481] is done
--> get similarity of txt[11482] is done
--> get similarity of txt[11483] is done
--> get similarity of txt[11484] is done
--> get similarity of txt[11485] is done
--> get similarity of txt[11486] is done
--> get similarity of txt[11487] is done
--> get similarity of txt[11488] is done
--> get similarity of txt[11489] is done
--> get similarity of txt[11490] is done
--> get similarity of txt[11491] is done
--> get similarity of txt[11492] is done
--> get similarity of txt[11493] is done
--> get similarity of txt[11494] is done
--> get similarity of txt[11495] is done
--> get similarity of txt[11496] is done
--> get similari

--> get similarity of txt[11673] is done
--> get similarity of txt[11674] is done
--> get similarity of txt[11675] is done
--> get similarity of txt[11676] is done
--> get similarity of txt[11677] is done
--> get similarity of txt[11678] is done
--> get similarity of txt[11679] is done
--> get similarity of txt[11680] is done
--> get similarity of txt[11681] is done
--> get similarity of txt[11682] is done
--> get similarity of txt[11683] is done
--> get similarity of txt[11684] is done
--> get similarity of txt[11685] is done
--> get similarity of txt[11686] is done
--> get similarity of txt[11687] is done
--> get similarity of txt[11688] is done
--> get similarity of txt[11689] is done
--> get similarity of txt[11690] is done
--> get similarity of txt[11691] is done
--> get similarity of txt[11692] is done
--> get similarity of txt[11693] is done
--> get similarity of txt[11694] is done
--> get similarity of txt[11695] is done
--> get similarity of txt[11696] is done
--> get similari

--> get similarity of txt[11873] is done
--> get similarity of txt[11874] is done
--> get similarity of txt[11875] is done
--> get similarity of txt[11876] is done
--> get similarity of txt[11877] is done
--> get similarity of txt[11878] is done
--> get similarity of txt[11879] is done
--> get similarity of txt[11880] is done
--> get similarity of txt[11881] is done
--> get similarity of txt[11882] is done
--> get similarity of txt[11883] is done
--> get similarity of txt[11884] is done
--> get similarity of txt[11885] is done
--> get similarity of txt[11886] is done
--> get similarity of txt[11887] is done
--> get similarity of txt[11888] is done
--> get similarity of txt[11889] is done
--> get similarity of txt[11890] is done
--> get similarity of txt[11891] is done
--> get similarity of txt[11892] is done
--> get similarity of txt[11893] is done
--> get similarity of txt[11894] is done
--> get similarity of txt[11895] is done
--> get similarity of txt[11896] is done
--> get similari

--> get similarity of txt[12073] is done
--> get similarity of txt[12074] is done
--> get similarity of txt[12075] is done
--> get similarity of txt[12076] is done
--> get similarity of txt[12077] is done
--> get similarity of txt[12078] is done
--> get similarity of txt[12079] is done
--> get similarity of txt[12080] is done
--> get similarity of txt[12081] is done
--> get similarity of txt[12082] is done
--> get similarity of txt[12083] is done
--> get similarity of txt[12084] is done
--> get similarity of txt[12085] is done
--> get similarity of txt[12086] is done
--> get similarity of txt[12087] is done
--> get similarity of txt[12088] is done
--> get similarity of txt[12089] is done
--> get similarity of txt[12090] is done
--> get similarity of txt[12091] is done
--> get similarity of txt[12092] is done
--> get similarity of txt[12093] is done
--> get similarity of txt[12094] is done
--> get similarity of txt[12095] is done
--> get similarity of txt[12096] is done
--> get similari

--> get similarity of txt[12273] is done
--> get similarity of txt[12274] is done
--> get similarity of txt[12275] is done
--> get similarity of txt[12276] is done
--> get similarity of txt[12277] is done
--> get similarity of txt[12278] is done
--> get similarity of txt[12279] is done
--> get similarity of txt[12280] is done
--> get similarity of txt[12281] is done
--> get similarity of txt[12282] is done
--> get similarity of txt[12283] is done
--> get similarity of txt[12284] is done
--> get similarity of txt[12285] is done
--> get similarity of txt[12286] is done
--> get similarity of txt[12287] is done
--> get similarity of txt[12288] is done
--> get similarity of txt[12289] is done
--> get similarity of txt[12290] is done
--> get similarity of txt[12291] is done
--> get similarity of txt[12292] is done
--> get similarity of txt[12293] is done
--> get similarity of txt[12294] is done
--> get similarity of txt[12295] is done
--> get similarity of txt[12296] is done
--> get similari

In [115]:
relations2 = pd.DataFrame(relations)

In [116]:
relations2

,0,1,2,3,4,5,6,7,8,9,...,12422,12423,12424,12425,12426,12427,12428,12429,12430,12431
0,1.000000,0.506758,0.732986,0.535308,0.535308,0.470881,0.485150,0.581003,0.581003,0.581003,...,0.485250,0.457370,0.457370,0.519819,0.467235,0.541032,0.541032,0.475303,0.475303,0.475303
1,0.506758,1.000000,0.620986,0.655690,0.655690,0.619371,0.665998,0.587185,0.587185,0.587185,...,0.572627,0.548841,0.548841,0.624316,0.657973,0.627519,0.627519,0.651099,0.651099,0.651099
2,0.732986,0.620986,1.000000,0.623184,0.623184,0.548980,0.621620,0.623090,0.623090,0.623090,...,0.619057,0.616934,0.616934,0.642877,0.620950,0.645736,0.645736,0.648143,0.648143,0.648143
3,0.535308,0.655690,0.623184,1.000000,1.000000,0.670929,0.766967,0.641585,0.641585,0.641585,...,0.650434,0.670929,0.670929,0.669601,0.676255,0.678070,0.678070,0.665702,0.665702,0.665702
4,0.535308,0.655690,0.623184,1.000000,1.000000,0.670929,0.766967,0.641585,0.641585,0.641585,...,0.650434,0.670929,0.670929,0.669601,0.676255,0.678070,0.678070,0.665702,0.665702,0.665702
5,0.470881,0.619371,0.548980,0.670929,0.670929,1.000000,0.705188,0.540150,0.540150,0.540150,...,0.519902,0.489033,0.489033,0.556530,0.651833,0.559768,0.559768,0.549953,0.549953,0.549953
6,0.485150,0.665998,0.621620,0.766967,0.766967,0.705188,1.000000,0.678009,0.678009,0.678009,...,0.632424,0.629684,0.629684,0.646709,0.660273,0.599256,0.599256,0.663700,0.663700,0.663700
7,0.581003,0.587185,0.623090,0.641585,0.641585,0.540150,0.678009,1.000000,1.000000,1.000000,...,0.625444,0.620376,0.620376,0.685420,0.674312,0.689979,0.689979,0.694179,0.694179,0.694179
8,0.581003,0.587185,0.623090,0.641585,0.641585,0.540150,0.678009,1.000000,1.000000,1.000000,...,0.625444,0.620376,0.620376,0.685420,0.674312,0.689979,0.689979,0.694179,0.694179,0.694179
9,0.581003,0.587185,0.623090,0.641585,0.641585,0.540150,0.678009,1.000000,1.000000,1.000000,...,0.625444,0.620376,0.620376,0.685420,0.674312,0.689979,0.689979,0.694179,0.694179,0.694179


In [117]:
relations2.to_csv('/home/pirl/Downloads/document_clustering/txt_similarity.csv',mode="w",index=True)